In [13]:
import datetime
import requests
import json
import numpy as np
import pickle
import os
import folium
from branca.element import Template, MacroElement
import math
from shapely.geometry import MultiPoint, Polygon
from shapely.ops import unary_union, polygonize
import alphashape

In [14]:
# qui funzione per calcolare distanza in linea d'area
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Raggio della Terra in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

In [15]:
# importiamo la griglia di punti della città
with open('./1500_città_metropolitana_milano.json', 'r') as file:
    griglia = json.load(file)

In [16]:
# importiamo i punti di interesse (teatri, cinema, ecc)
with open('./punti_di_interesse/biblioteche_città_metropolitana_milano.json', 'r') as file:
    teatri = json.load(file)

In [17]:
# passiamo in un formato più comodo
teatri_n = []
for dic in teatri:
    coords = [dic['latitude'],dic['longitude']]
    teatri_n.append(coords)

In [18]:
# tanto per stare sicuri visualizziamo cose
center = [45.4642, 9.1900]

mappa = folium.Map(location=center, zoom_start=13)

for point in teatri_n:
    folium.Marker(location=point).add_to(mappa)

mappa

In [19]:
# # richiesta API
# API_KEY = 'YOUR_API_KEY'
# base_url = 'https://maps.googleapis.com/maps/api/distancematrix/json'

# data = []

# # parametri
# departure_time = 1725472800  # timestamp in UNIX per il tempo di partenza
# mode = 'transit'  # modalità di viaggio

# # iterazione
# for punto in griglia:
#     # qui calc oliamo distanze in linea d'aria per ogni punto di interesse
#     distanze = []
#     for teatro in teatri_n:
#         distanza = haversine(punto[0], punto[1], teatro[0], teatro[1])
#         distanze.append((teatro, distanza))
    
#     # riodiniamo per distanza e selezioniamo i primi 10 punti di interesse
#     distanze.sort(key=lambda x: x[1])
#     teatri_piu_vicini = distanze[:10]
    
#     origins = f"{punto[0]},{punto[1]}"
#     destinations = "|".join([f"{teatro[0][0]},{teatro[0][1]}" for teatro in teatri_piu_vicini])
    
    
#     params = {
#         'key': API_KEY,
#         'origins': origins,
#         'destinations': destinations,
#         'departure_time': departure_time,
#         'mode': mode
#     }
    
#     # richiesta
#     r = requests.get(base_url, params=params)
    
    
#     if r.status_code == 200:
        
#         response_data = r.json()
        
        
#         data.append({
#             'origins': origins,
#             'destinations': destinations,
#             'results': response_data
#         })
#     else:
#         print(f"errore con la richiesta per il punto {origins}: {r.status_code}")

# # salviamo in un file JSON
# with open('tempi_di_viaggio_biblioteche_cmm.json', 'w') as f:
#     json.dump(data, f, indent=4)

In [20]:

with open('./tempi di viaggio/tempi_di_viaggio_biblioteche_cmm.json', 'r') as file:
    data = json.load(file)

In [21]:
formatted_results = []

for result in data:
    origin = result['origins']
    durations = []
    for element in result['results']['rows'][0]['elements']:
        if 'duration' in element:
            durations.append(element['duration']['value'])
        else:
            durations.append(None)  

    formatted_results.append([origin] + durations)

In [22]:
coords_nresults = []
for item in formatted_results:
    # estraiamo le coordinate e i valori di durata
    coordinates = item[0]
    durations = item[1:]
    
    # contiamo quanti valori sono sotto un tempo in secondi, escludendo i None
    count_under_t = sum(1 for duration in durations if duration is not None and duration < 1800)
    
    # creaimo lista di tuple con coordinate e risultato
    coords_nresults.append((coordinates, count_under_t))


## La mappa in leaflet

In [30]:
import pandas as pd
import geopandas as gpd
import folium
import numpy as np
from shapely.geometry import Polygon, Point
import branca.colormap as cm
import matplotlib.colors as mcolors
from branca.element import Template, MacroElement, JavascriptLink, CssLink

# coordinate del centro di Milano
center = [45.4642, 9.1900]

# calcoliamo i valori minimo e massimo
min_value =0
max_value =4

# Create the initial DataFrame
df = pd.DataFrame()
geometry, values = [], []
for coords, value in coords_nresults:
    coords = coords.split(',')
    geometry.append(coords)
    values.append(value)

df["geometry"] = geometry
df["values"] = values

# Create points from coordinates
points = gpd.points_from_xy(df["geometry"].str[1].astype(float), df["geometry"].str[0].astype(float))
gdf = gpd.GeoDataFrame(df, geometry=points, crs="EPSG:4326")

def create_custom_colormap():
    colors = ['blue', 'grey', 'red']
    n_bins = 100  # Number of color gradations
    cmap = mcolors.LinearSegmentedColormap.from_list("custom", colors, N=n_bins)
    return cmap

custom_cmap = create_custom_colormap()
colormap = cm.LinearColormap(
    colors=[mcolors.rgb2hex(custom_cmap(i)) for i in np.linspace(0, 1, 256)],
    vmin=min_value,
    vmax=max_value
)

def hex_coordinates(center, size):
    x, y = center
    h = size * 2
    w = np.sqrt(3) * size
    return [
        (x, y + h/2),
        (x + w/2, y + h/4),
        (x + w/2, y - h/4),
        (x, y - h/2),
        (x - w/2, y - h/4),
        (x - w/2, y + h/4)
    ]

def create_hex_grid(bounds, size):
    minx, miny, maxx, maxy = bounds
    h = size * 2
    w = np.sqrt(3) * size
    
    x_coords = np.arange(minx, maxx + w, w)
    y_coords = np.arange(miny, maxy + h, h * 3/4)
    
    hexagons = []
    for i, y in enumerate(y_coords):
        offset = (w / 2) if i % 2 else 0
        for x in x_coords:
            hexagons.append(Polygon(hex_coordinates((x + offset, y), size)))
    
    return hexagons

def create_hex_gdf(gdf):
    bounds = gdf.total_bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    n = len(gdf)
    hex_size = np.sqrt((width * height) / (n * 2 * np.sqrt(3)))

    hexagons = create_hex_grid(bounds, hex_size)
    hex_gdf = gpd.GeoDataFrame(geometry=hexagons, crs=gdf.crs)
    
    # Join with original points
    joined = gpd.sjoin(hex_gdf, gdf, how="left", predicate="contains")
    
    # In case of multiple points in a hexagon, take the mean of values
    result = joined.groupby(joined.index).agg({'values': 'mean', 'geometry': 'first'})
    
    # Make sure result is a GeoDataFrame
    result = gpd.GeoDataFrame(result, geometry='geometry', crs=gdf.crs)
    
    # get rid of NaN values
    result.dropna(subset=['values'], inplace=True)
    
    return result

# Create the hexagonal grid
hex_gdf = create_hex_gdf(gdf)

In [31]:


# Create the base map with Positron without labels
m = folium.Map(location=center, zoom_start=11.4, tiles="cartodbpositronnolabels")

# Add the GeoJSON layer with choropleth style
choropleth = folium.GeoJson(
    hex_gdf.__geo_interface__,
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']["values"]) if feature['properties']["values"] is not None else 'gray',
        'color': 'white',
        'weight': 0.1,
        'fillOpacity': 0.3,
    },
    name='Choropleth'
).add_to(m)


# Add the Scomodo logo
logo_html = '''
    <div style="
        position: fixed; 
        bottom: 10px; 
        right: 10px; 
        width: 150px; 
        height: 50px; 
        z-index:9999;">
        <a href="https://scomodo.org" target="_blank">
            <img src="https://scomodo.org/wp-content/uploads/2023/04/logo-scomodo-black.png" style="width:100%; height:100%; object-fit: contain;">
        </a>
    </div>
'''

# Add the logo to the map
m.get_root().html.add_child(folium.Element(logo_html))

# Create a custom legend with gradient
legend_html = '''
{% macro html(this, kwargs) %}
<div id="legend" style="
    position: fixed; 
    top: 120px;
    left: 50px;
    width: 220px;
    height: 100px;
    z-index:9999; 
    font-size:14px;
    background-color: rgba(255,255,255,0.8);
    border-radius: 5px;
    box-shadow: 0 0 15px rgba(0,0,0,0.2);
    padding: 10px;
    ">
    <div style="margin-bottom: 5px;"><strong>Punti di interesse</strong></div>
    <div style="display: flex; height: 20px; margin-bottom: 5px;">
        <div style="flex-grow: 1; background: linear-gradient(to right, blue, grey, red);"></div>
    </div>
    <div style="display: flex; justify-content: space-between;">
        <span>{{ this.min }}</span>
        <span>{{ (this.min + this.max) / 2 | round | int }}</span>
        <span>{{ this.max }}</span>
    </div>
</div>
{% endmacro %}
'''

legend = MacroElement()
legend._template = Template(legend_html)
legend.min = int(min_value)
legend.max = int(max_value)
m.get_root().add_child(legend)

# Add a well-formatted title
title_html = '''
    <div id="title" style="
        position: fixed; 
        top: 10px; 
        left: 50px; 
        width: 500px;
        z-index:9999; 
        font-size: 24px; 
        font-weight: bold;
        background-color: rgba(255,255,255,0.8);
        border-radius: 5px;
        padding: 10px;
        box-shadow: 0 0 15px rgba(0,0,0,0.2);">
        <h3 style="margin-bottom: 5px;">Mappa dei Punti di Interesse</h3>
        <p style="font-size: 16px; margin: 0;">Numero di punti di interesse raggiungibili in meno di 30 minuti</p>
    </div>
'''
m.get_root().html.add_child(folium.Element(title_html))


# Add labels
folium.map.CustomPane("labels").add_to(m)

# Final layer associated to custom pane via the appropriate kwarg
folium.TileLayer("cartodbpositrononlylabels", pane="labels").add_to(m)


# Save the map
m.save('milan_hexagon_choropleth_map.html')
m